In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-quiz2-4/train.csv
/kaggle/input/2021-ml-quiz2-4/test.csv
/kaggle/input/2021-ml-quiz2-4/sample_submit.csv


In [2]:
#데이터 로드
train = pd.read_csv('../input/2021-ml-quiz2-4/train.csv')
test = pd.read_csv('../input/2021-ml-quiz2-4/test.csv')
submit = pd.read_csv('../input/2021-ml-quiz2-4/sample_submit.csv')

In [3]:
train.head()

,index,CRIM,INDUS,NOX,RM,LSTAT,B,PTRATIO,ZN,CHAS,AGE,RAD,DIS,TAX,Price
0,0,0.14150,0.0,6.91,0,0.448,6.169,6.6,5.7209,3,233,17.9,383.37,5.81,25.3
1,1,0.15445,25.0,5.13,0,0.453,6.145,29.2,7.8148,8,284,19.7,390.68,6.86,23.3
2,2,16.81180,0.0,18.10,0,0.700,5.277,98.1,1.4261,24,666,20.2,396.90,30.81,7.2
3,3,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398,18.7,386.40,12.34,21.2
4,4,8.79212,0.0,18.10,0,0.584,5.565,70.6,2.0635,24,666,20.2,3.65,17.16,11.7


In [4]:
#학습데이터 및 라벨 생성
x_train = train.drop(['index','Price'],axis=1)
x_test = test.drop(['index'],axis=1)
y_train = train['Price']

In [33]:
from sklearn.preprocessing import RobustScaler
ro = RobustScaler()
x_train_std = ro.fit_transform(x_train)
x_test_std = ro.transform(x_test)

In [15]:
#autosklearn 설치
! apt-get install build-essential swig
! pip3 install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 1100 kB of archives.
After this operation, 5822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6460 B]
Fetched 1100 kB in 0s (4962 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package swig3.0.
(Reading database ... 95327 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package s

In [22]:
#estimators 확인
from autosklearn.pipeline.components import regression
regression._regressors.keys()
model = ['adaboost','decision_tree', 'k_nearest_neighbors','libsvm_svr','random_forest']

In [40]:
#AutoSklearnRegressor 실행
from autosklearn.regression import AutoSklearnRegressor
automl = AutoSklearnRegressor(
    time_left_for_this_task=300,
    per_run_time_limit=60,
    include_estimators=model,
    n_jobs = -1   
)
automl.fit(x_train_std, y_train)

/opt/conda/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42307 instead
  http_address["port"], self.http_server.port


AutoSklearnRegressor(include_estimators=['adaboost', 'decision_tree',
                                         'k_nearest_neighbors', 'libsvm_svr',
                                         'random_forest'],
                     n_jobs=-1, per_run_time_limit=60,
                     time_left_for_this_task=300)

In [41]:
#best모델 확인
data = pd.DataFrame(automl.cv_results_)
data['params'][data['rank_test_scores'].sort_values().index[0]]

{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'median',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'minmax',
 'feature_preprocessor:__choice__': 'polynomial',
 'regressor:__choice__': 'libsvm_svr',
 'feature_preprocessor:polynomial:degree': 2,
 'feature_preprocessor:polynomial:include_bias': 'True',
 'feature_preprocessor:polynomial:interaction_only': 'False',
 'regressor:libsvm_svr:C': 180.22479438529933,
 'regressor:libsvm_svr:epsilon': 0.0012571604901280202,
 'regressor:libsvm_svr:kernel': 'rbf',
 'regressor:libsvm_svr:max_iter': -1,
 'regressor:libsvm_svr:shrinking': 'False',
 'regressor:libsvm_svr:tol': 0.02945259690926852,
 'regressor:libsvm_svr:degree': 4,
 'regressor:libsvm_svr:gamma': 0.10272820821863678}

In [42]:
data[data['rank_test_scores']==1]

,mean_test_score,mean_fit_time,params,rank_test_scores,status,budgets,param_data_preprocessing:categorical_transformer:categorical_encoding:__choice__,param_data_preprocessing:categorical_transformer:category_coalescence:__choice__,param_data_preprocessing:numerical_transformer:imputation:strategy,param_data_preprocessing:numerical_transformer:rescaling:__choice__,...,param_feature_preprocessor:fast_ica:n_components,param_feature_preprocessor:kernel_pca:coef0,param_feature_preprocessor:kernel_pca:degree,param_feature_preprocessor:kernel_pca:gamma,param_feature_preprocessor:nystroem_sampler:coef0,param_feature_preprocessor:nystroem_sampler:degree,param_feature_preprocessor:nystroem_sampler:gamma,param_regressor:libsvm_svr:coef0,param_regressor:libsvm_svr:degree,param_regressor:libsvm_svr:gamma
16,0.837411,1.347305,{'data_preprocessing:categorical_transformer:c...,1,Success,0.0,no_encoding,no_coalescense,median,minmax,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.102728


In [43]:
#예측 및 저장
pred = automl.predict(x_test_std)
submit['price'] = pred
submit.to_csv('submit#4.csv', index=False)
submit

,index,price
0,0,30.326537
1,1,24.626143
2,2,17.913420
3,3,21.080374
4,4,24.153348
...,...,...
97,97,24.039016
98,98,40.275222
99,99,22.591995
100,100,15.366556


In [44]:
#모델 저장
from joblib import dump, load
dump(automl, 'automl#4.joblib') 

['automl#4.joblib']

필요한 데이터 추출 후 autoML을 통해 학습 및 예측을 진행하고 마지막으로 해당 모델을 저장했습니다.